# Omost - LLM + Canvas (Regional Prompting) + Stable Diffusion

In [ ]:
# @title ## Clone the repo and install requirements- Omost { display-mode: "form" }

!git clone https://github.com/lllyasviel/Omost.git
%cd Omost
#!conda create -n omost python=3.10
#!conda activate omost
#!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

#!python gradio_app.py


In [ ]:
# @title ## Edit the files { display-mode: "form" }

# function to edit the files
def Edit_file_code(goal, target_file, target_code, new_code, lines_offset, pop_x_lines):

    print(f'''-------------------------------------
Purpose: {goal}
target file: {target_file}
target code:\n{target_code}
new code:\n{new_code}
lines offset: {lines_offset}
{pop_x_lines} lines to pop''')

    # check if the edit was already done, exit if so
    with open(target_file, 'r') as file:
        data = file.read()
    if (new_code in data):
        print (f'''The new code was found in target file: {target_file}. The edit was already done. Exiting.''')
        return

    # read the file
    with open(target_file, 'r') as file:
        lines = file.readlines()

    found_line_number = -1

    # find the target line "target_code"
    for i, line in enumerate(lines):
        if target_code in line:
            found_line_number = i
            break

    # it should always find it, but whatever, insert "new_code" to the data
    if found_line_number != -1:
        if (pop_x_lines):
            print ("haa")
            for i in range(pop_x_lines):
                lines.pop(found_line_number)
        lines.insert(found_line_number + lines_offset, new_code)
        print (f'''New code added to target file: {target_file}. File edited.''')
    else:
        print (f'''Could not find target code: {target_code}.''')

    # rewrite target_file
    with open(target_file, 'w') as file:
        file.writelines(lines)


# functions for every edit/changes to be made

# to fix an error, can't remember what it was anymore, it works
def edit_memory_management_py_first_change():
    # variables
    goal = "First change"
    target_file = "/content/Omost/lib_omost/memory_management.py"
    target_code = '''    if not high_vram:'''
    new_code = r'''    if not high_vram:
        for m in models_to_unload:
            if m.__class__.__name__ != "LlamaForCausalLM":
                with movable_bnb_model(m):
                    m.to(cpu)
                    print('Unload to CPU:', m.__class__.__name__)
        models_in_gpu = models_to_remain
'''
    lines_offset = 0
    pop_x_lines = 6
    # call the edit file function
    Edit_file_code(goal, target_file, target_code, new_code, lines_offset, pop_x_lines)

# to fix an error, something about 2 devices, this sets it to gpu (I think? can't remember)
def edit_gradio_app_py_first_change():
    # variables
    goal = "First change"
    target_file = "/content/Omost/gradio_app.py"
    target_code = '''import os'''
    new_code = r'''import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
'''
    lines_offset = 0
    pop_x_lines = 1
    # call the edit file function
    Edit_file_code(goal, target_file, target_code, new_code, lines_offset, pop_x_lines)

# to get a gradio public/share link
def edit_gradio_app_py_share_link():
    # variables
    goal = "First change"
    target_file = "/content/Omost/gradio_app.py"
    target_code = '''    demo.queue().launch(inbrowser=True, server_name='0.0.0.0')'''
    new_code = r'''    demo.queue().launch(inbrowser=True, share=True, server_name='0.0.0.0')
'''
    lines_offset = 0
    pop_x_lines = 1
    # call the edit file function
    Edit_file_code(goal, target_file, target_code, new_code, lines_offset, pop_x_lines)


# to fix an error, This was supposed to unload all models to prevent OOM, doesn't work, kills everything instead when generating the image
def edit_memory_management_py_second_change_mine():
    # variables
    goal = "Second change"
    target_file = "/content/Omost/lib_omost/memory_management.py"
    target_code = '''def unload_all_models(extra_models=None):'''
    new_code = r'''def unload_all_models(extra_models=None):
    global models_in_gpu

    if extra_models is None:
        extra_models = []

    if not isinstance(extra_models, (tuple, list)):
        extra_models = [extra_models]

    models_in_gpu = list(set(models_in_gpu + extra_models))

    for m in models_in_gpu:
        with movable_bnb_model(m):
            m.to(cpu)
            print('Unload from GPU:', m.__class__.__name__)
            # Unload from CPU
            del m
    models_in_gpu = []
    torch.cuda.empty_cache()
    
    return load_models_to_gpu([])
'''
    lines_offset = 0
    pop_x_lines = 12
    # call the edit file function
    Edit_file_code(goal, target_file, target_code, new_code, lines_offset, pop_x_lines)



# call the changes
# These changes fix the first 2 errors, but it overloads the memory, both ram and vram
edit_memory_management_py_first_change()
edit_gradio_app_py_first_change()
# gradio public link
edit_gradio_app_py_share_link()

# This is supposed to unload all models to prevent OOM, doesn't work, kills the app
# suggested by you.com, goes to show what it knows
#edit_memory_management_py_second_change_mine()

In [ ]:
!python gradio_app.py